## Depth Contours

### Generate depth-contours of all the Pareto-optimal data sets.

This notebook can be used to generate tradeoff values from all the Pareto-optimal data point files hard-coded in the dictionary `pfs`. Currently this notebook processes these Pareto-optimal fronts.

- DTLZ2 ($m$-Sphere) Problem
- DEBMDK (Knee) Problem
- CDEBMDK (Constrained Knee) Problem
- C0-DTLZ2 (A split $𝑚$-sphere with a small isolated cluster at $f_m$-axis)
- C2-DTLZ2 Problem
- DTLZ8 Problem (A 3-dimensional line and an  𝑚 -dimensional hypersurface)
- GAA Problem (A 10-objective and 18-constraint general aviation design problem)

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams.update({'figure.max_open_warning': 0})

### Generate and save the depth contours

In this case we are computing the depth-contours from the convex-hulls. So we are using the `tda.simple_shape` module.

In [ ]:
sys.path.append('../')
from vis.tda import simple_shape
from vis.utils import io

pfs = {'dtlz2': ['3d', '4d', '8d'], \
       'dtlz2-nbi': ['3d', '4d', '8d'], \
       'debmdk': ['3d', '4d', '8d'], \
       'debmdk-nbi': ['3d', '4d', '8d'], \
       'debmdk-all': ['3d', '4d', '8d'], \
       'debmdk-all-nbi': ['3d', '4d', '8d'], \
       'dtlz8': ['3d', '4d', '6d', '8d'], \
       'dtlz8-nbi': ['3d', '4d', '6d', '8d'], \
       'c2dtlz2': ['3d', '4d', '5d', '8d'], \
       'c2dtlz2-nbi': ['3d', '4d', '5d', '8d'], \
       'cdebmdk': ['3d', '4d', '8d'], \
       'cdebmdk-nbi': ['3d', '4d', '8d'], \
       'c0dtlz2': ['3d', '4d', '8d'], \
       'c0dtlz2-nbi': ['3d', '4d', '8d'], \
       'crash-nbi': ['3d'], 'crash-c1-nbi': ['3d'], 'crash-c2-nbi': ['3d'], \
       'gaa': ['10d'], \
       'gaa-nbi': ['10d']}

for pf in list(pfs.keys())[17:]:
    for dim in pfs[pf]:
        fullpathf = "../data/{0:s}/{1:s}/dataf.csv".format(pf, dim)
        if os.path.exists(fullpathf):
            path, filenamef = os.path.split(fullpathf)
            dirs = path.split('/')
            frontname = dirs[-2]

            F = np.loadtxt(fullpathf, delimiter=',')
            print(fullpathf, F.shape, dirs, frontname)
            
            # test simple_shape.depth_contour function
            # it looks like these PFs are better displayed if project_collapse=False
            if pf in ['dtlz8', 'dtlz8-nbi', 'crash-nbi', 'crash-c1-nbi', 'crash-c2-nbi']:
                L = simple_shape.depth_contours(F, project_collapse=False)
            elif pf in ['gaa', 'gaa-nbi']:
                L = simple_shape.depth_contours(F, verbose=True)
            else:
                L = simple_shape.depth_contours(F)
            # save the layers
            io.savetxt(os.path.join(path, "depth-cont-cvhull.csv"), L, fmt='{:d}', delimiter=',')
            
            # We are not using this since it's exrtemely slow and also doesn't give
            # layers if all the points are on a fully convex surface.
            # print("Generating depth-contours (project_collapse=False) for " + frontname)
            # # test ss.depth_contour function without projection and collapse
            # L = ss.depth_contours(F, project_collapse = False)
            # save the layers
            # io.savetxt(os.path.join(path, "depth-cont-cvhull.csv"), L, fmt = '{:d}', delimiter = ',')

../data/gaa/10d/dataf.csv (3112, 10) ['..', 'data', 'gaa', '10d'] gaa
Projecting on a simplex and collapsing.
Computing depth contour 0 ...


### Plot and verify

In [ ]:
%matplotlib inline

sys.path.append('../')
from vis.utils import io

for pf in list(pfs.keys()):
    for dim in pfs[pf]:
        fullpathf = "../data/{0:s}/{1:s}/dataf.csv".format(pf, dim)
        if os.path.exists(fullpathf):
            path, filenamef = os.path.split(fullpathf)
            dirs = path.split('/')
            frontname = dirs[-2]
            
            F = np.loadtxt(fullpathf, delimiter = ',')
            print(fullpathf, F.shape, dirs, frontname)
            
            layerpathf = os.path.join(path, "depth-cont-cvhull.csv")
            if os.path.exists(layerpathf):
                L = io.loadtxt(layerpathf, dtype=int, delimiter=',')
                if F.shape[1] == 2:
                    fig = plt.figure()
                    ax = fig.gca()
                    for l in L:
                        ax.scatter(F[l.astype(int),0], F[l.astype(int),1], s=1)
                    plt.show()
                else:
                    fig = plt.figure()
                    ax = Axes3D(fig)
                    for l in L:
                        ax.scatter(F[l.astype(int),0], F[l.astype(int),1], F[l.astype(int),2], s=1)
                    plt.show()